In [1]:
from qtinflux import client
import pandas as pd

In [2]:
#get proejcts information
rs = client.query("SHOW TAG VALUES FROM dependencies WITH KEY=project")
nodes = pd.DataFrame(rs.get_points())
nodes = nodes.drop(columns=['key'])
nodes = nodes.append({'value': 'qtbase'}, ignore_index=True)

projects = nodes['value'].tolist()
projects.sort()


In [3]:
def getGerritProjectList(projects):
    list = []
    for p in projects:
        list.append("qt/"+p)
    return list


testList = getGerritProjectList(projects)


In [4]:
from pygerrit2 import GerritRestAPI, HTTPBasicAuth
from credentials import gerritLogin
from datetime import datetime
from datetime import date

today = date.today()
print(today)


auth = HTTPBasicAuth(gerritLogin['user'], gerritLogin['password'])
rest = GerritRestAPI(url='https://codereview.qt-project.org/', auth=auth)
changes = rest.get("/changes/?q=owner:qt_submodule_update_bot@qt-project.org%20status:open")
print(len(changes))


2020-05-19
39


In [5]:
# this function returns a list with values who represet no of days since last successful update, read below for execeptions
# the order is same as the order of projects list
# there are 2 special reserved values: -2 for a projet who has no dependencies e.g. qt/qtbase
#                                      -1 for a conflict case when Qt Update Bot failed at updating the module    
def getUpdateBotValues(modulesList):    
    prefix  = "qt/"
    daysSinceLastUpdate = []
    for module in modulesList:    
        request = "/changes/?q=owner:qt_submodule_update_bot@qt-project.org%20project:" + prefix + module + "%20branch:dev"
        changes = rest.get(request)        
        if (len(changes) > 0):
            # print projet name:
            print(changes[0]['project']) 
            updateString = changes[0]['updated'][:-19]            
            updateDate = datetime.strptime(updateString, '%Y-%m-%d').date()
            print(updateDate)                  
            howManyDays = today - updateDate
            print("howManyDays", howManyDays.days)
            change = changes[0]
            try:
                if change['mergeable']:                
                    print("Conflict!!!!")
                    daysSinceLastUpdate.append(-1)
            except KeyError:           
                pass    
                daysSinceLastUpdate.append(howManyDays.days)
            print("-----------------")
        else:
            daysSinceLastUpdate.append(-2)
    return daysSinceLastUpdate

#getUpdateBotConflicts(getUpdateBotConflicts)     

In [6]:
projectsUpdatesStatus = getUpdateBotValues(projects)  

qt/qtandroidextras
2020-05-15
howManyDays 4
-----------------
qt/qtdeclarative
2020-05-16
howManyDays 3
-----------------
qt/qtdoc
2020-05-08
howManyDays 11
Conflict!!!!
-----------------
qt/qtgraphicaleffects
2020-05-17
howManyDays 2
-----------------
qt/qtimageformats
2020-05-15
howManyDays 4
-----------------
qt/qtnetworkauth
2020-05-15
howManyDays 4
-----------------
qt/qtquick3d
2020-05-17
howManyDays 2
-----------------
qt/qtquickcontrols2
2020-05-18
howManyDays 1
-----------------
qt/qtquicktimeline
2020-05-17
howManyDays 2
-----------------
qt/qtshadertools
2020-05-15
howManyDays 4
-----------------
qt/qtsvg
2020-05-15
howManyDays 4
-----------------
qt/qttools
2020-05-18
howManyDays 1
-----------------
qt/qttranslations
2020-05-06
howManyDays 13
-----------------
qt/qtvirtualkeyboard
2020-05-17
howManyDays 2
Conflict!!!!
-----------------
qt/qtwayland
2020-05-17
howManyDays 2
-----------------
qt/qtwebsockets
2020-05-18
howManyDays 1
Conflict!!!!
-----------------


In [7]:
#get links information
rs = client.query("SELECT * FROM dependencies")
links = pd.DataFrame(rs.get_points())
links = links.drop(columns=['branch', 'depHeadSha1', 'time'])
links = links.rename(columns={"project": "source", "dependency": "target", "howFar": "value"})
#order change: i want an obvious order: source->target, value, where sorce is project, target is dependency, value is no of commits
links = links[['source', 'target','value']]

In [8]:
def generateXList(list):
    out = []
    for l in list:
        for i in range(len(list)):        
            out.append(l)
    return out

In [9]:
def generateYList(list):
    out = []
    for i in range(len(list)):
        for l in list:
            out.append(l)
    return out

In [10]:
def generateText(list):
    out = []
    count = 0
    for j in list:
        for i in list: 
            df = links.loc[links['source'] == i]
            df = df.loc[df['target'] == j]
            df = df.reset_index()
            if df.empty:
                out.append('')
            else:              
                out.append(df.iloc[0]['value'])
            count += 1
    return out

In [11]:
def generateIndexes(list):
    out = []
    count = 0
    for j in list:
        for i in list: 
            df = links.loc[links['source'] == i]
            df = df.loc[df['target'] == j]
            df = df.reset_index()
            if not df.empty:                                            
                out.append(count)
            count += 1
    return out

In [12]:
from bokeh.colors.groups import black
from bokeh.colors.groups import white
from bokeh.colors.groups import cyan

other = 'lightcyan'
def generateColors(list):
    out = []
    for i in range(len(list)):       
        nextCol = ""
        for j in range(len(list)):
            if nextCol == "":
                if i % 2 == 0:
                    color = "white"            
                    nextCol = other 
                else:
                    color = other 
                    nextCol = "white"
            else:
                color = nextCol
                if nextCol == "white":
                    nextCol = other 
                else:
                    nextCol = "white"            
            out.append(color)
    return out


In [13]:
def generateJS(list):
    indexes = generateIndexes(list)
    
    out = "switch (cb_data.index.indices[0]) {"
    for i in indexes:
        out += " case "+str(i)+ ": cb_obj.tooltips = [['qt module', '@y'],['info', '@text'],['dependency', '@x']]; break;"
    out += " default: cb_obj.tooltips=[] }"
    return out
                   
#code = generateJS(projects)
#print(code)

In [14]:
#[3, -2, 3, -1, 3, 3, 3, 3, 3, 3, 3, -1, 5, 3, 3, 3]
#returns min and max of list with how many days passed, ignores reserved values -1 and -2
def getExtremes(list):
    max = 0
    min = 0
    for e in list:
        if e >= 0:
            if e > max:
                max = e
            if e < min:                
                min = e            
    return (min,max)        
    

In [15]:
#[3, -2, 3, -1, 3, 3, 3, 3, 3, 3, 3, -1, 5, 3, 3, 3]
#returns min and max of list with how many days passed, ignores reserved values -1 and -2
def getExtremes(list):
    min, max = list[0],list[0]    
    for e in list:
        if e >= 0:
            if e > max:
                max = e
            if e < min:                
                min = e            
    return (min,max)        
    

In [16]:
palette = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1"]

In [17]:
def colorMap(extremes, value):    
    min = extremes[0]
    max = extremes[1]
    #print("min", min)
    #print("max", max)        
    delta = max-min
    if delta < len(palette):
     #   print("value", value)
      #  print("-------------------")        
        return palette[value-min]
    else:
        ratio = len(palette)/delta
       # print("ratio", ratio)
       # print("len of colors", len(colors))
       # print("int(value/ratio)", int(value/ratio))
        return palette[int(value/ratio)]        

In [18]:
def generateLegend(extremes, hm):
    min = extremes[0]
    max = extremes[1]    
    delta = max-min
    items = []
    #i = 1
    if delta < len(palette):
        i = 0
        for color in palette:        
            r = hm.square([0], [0], size=1, color=color)
            if min + i == 1:
                item_name = str(i + min)+ " day"
            else:
                item_name = str(i + min)+ " days"        
            items.append( (item_name, [r]))
            i += 1
    
    r = hm.square([0], [0], size=1, color="#cc7878")
    items.append(("Merge conflict", [r]))
    legend = Legend(items=items, location="center", title = "Days since SubmoduleUpdateBot run")
    return legend   

In [19]:
#[3, 3, -1, 3, 3, 3, 3, 3, 3, 3, -1, 5, 3, 3, 3]
#colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
#def colorMapping(max, )

def generateColors4(list):
    
    out = []
    #color = "white"
    for i in range(len(list)):        
        for j in range(len(list)):
            if list[j] == -1:
                out.append("#cc7878")                
            else:
                if list[j] == -2:
                    out.append("gray")                
                else:                    
                    out.append(colorMap(extremes,list[j]))
    return out

In [20]:
x = generateXList(projects)
y = generateYList(projects)

numbers = generateText(projects)
#colors = generateColors(projects)
projectsUpdatesStatus = getUpdateBotValues(projects)  
extremes = getExtremes(projectsUpdatesStatus)    
colors = generateColors4(projectsUpdatesStatus)

qt/qtandroidextras
2020-05-15
howManyDays 4
-----------------
qt/qtdeclarative
2020-05-16
howManyDays 3
-----------------
qt/qtdoc
2020-05-08
howManyDays 11
Conflict!!!!
-----------------
qt/qtgraphicaleffects
2020-05-17
howManyDays 2
-----------------
qt/qtimageformats
2020-05-15
howManyDays 4
-----------------
qt/qtnetworkauth
2020-05-15
howManyDays 4
-----------------
qt/qtquick3d
2020-05-17
howManyDays 2
-----------------
qt/qtquickcontrols2
2020-05-18
howManyDays 1
-----------------
qt/qtquicktimeline
2020-05-17
howManyDays 2
-----------------
qt/qtshadertools
2020-05-15
howManyDays 4
-----------------
qt/qtsvg
2020-05-15
howManyDays 4
-----------------
qt/qttools
2020-05-18
howManyDays 1
-----------------
qt/qttranslations
2020-05-06
howManyDays 13
-----------------
qt/qtvirtualkeyboard
2020-05-17
howManyDays 2
Conflict!!!!
-----------------
qt/qtwayland
2020-05-17
howManyDays 2
-----------------
qt/qtwebsockets
2020-05-18
howManyDays 1
Conflict!!!!
-----------------


IndexError: list index out of range

In [21]:
import math
from bokeh.plotting import figure, show, output_file
from bokeh.models import TapTool, HoverTool, OpenURL, CustomJS, ColumnDataSource,CustomJSHover, Rect, Title
from bokeh.models import Legend

hm = figure(tools="tap, reset",  
            toolbar_location="right",
            x_range=projects,
            x_axis_label = 'dependency',
            y_range=projects,
            y_axis_label = 'Qt module',            
            plot_width=1100,
            plot_height=900)

hm.xaxis.major_label_text_font_size = "12pt"
hm.xaxis.axis_label_text_font_size = "20pt"

hm.yaxis.major_label_text_font_size = "12pt"
hm.yaxis.axis_label_text_font_size = "20pt"

hm.add_layout(Title(text="Qt modules dependency map",
                   align="left",
                   text_color = "darkred",
                   text_font_size = "20px"),                 
                   "below")


hm.add_layout(Title(text="last update: ",
                   align="right",
                   text_color = "darkgray",
                   text_font_size = "15px"),                 
                   "below")


taptool = hm.select(type=TapTool)
taptool.callback = OpenURL(url="https://testresults.qt.io/grafana/d/pJyKbjXZz/git?orgId=1&var-project=@y")

tapCallback = CustomJS(args=dict(hm=hm), code='''hm.reset.emit();''') 
hm.js_on_event('tap', tapCallback)

rectGlyph = hm.rect(x, y, color = colors, hover_color="lightgray", line_color = "lightgray", width=1, height=1)
rectHover = HoverTool()
rectHover.tooltips=None
rectHover.renderers=[rectGlyph]
hm.add_tools(rectHover)

textGlyph =  hm.text(x, y, text=numbers, text_baseline = "middle", text_align="center")
textHover = HoverTool()
textHover.renderers=[textGlyph]
textHoverCode = ''' if (cb_data.index.indices.length > 0) 
{document.getElementsByClassName('bk-canvas-events')[0].style.cursor='pointer'} 
else {document.getElementsByClassName('bk-canvas-events')[0].style.cursor='default'}'''
textHover.tooltips = [["qtModule", "@y"], ["commits since last update", "@text"],["of submodule ", "@x"]]
textHover.callback = CustomJS(args = dict(source = numbers, hover = textHover), code = textHoverCode)
hm.add_tools(textHover)


output_file("index.html", title="qt modules dependencies map")

hm.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
hm.xaxis.axis_line_color = "pink"
hm.xaxis.major_label_orientation = math.pi/2 

hm.yaxis.axis_line_color = "pink"
hm.yaxis.major_tick_line_color = None  # turn off y-axis major ticks


legend = generateLegend(extremes,hm)
hm.add_layout(legend, 'right')





show(hm)  # open a browser


NameError: name 'colors' is not defined